In [7]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize


In [8]:
class DIDEstimator:
    def __init__(self, Y, D, T, X, W):
        self.Y = Y
        self.D = D
        self.T = T
        self.X = X
        self.W = W
        self.epsilon = 1e-10  # Small value to avoid numerical instability

    def probit(self, W, params):
        return norm.cdf(np.dot(W, params))
    
    def log_likelihood(self, params):
        p = np.clip(self.probit(self.W, params), self.epsilon, 1 - self.epsilon)
        return -np.sum(self.D * np.log(p) + (1 - self.D) * np.log(1 - p))
    
    def estimate_selection(self):
        initial_params = np.zeros(self.W.shape[1])
        result = minimize(self.log_likelihood, initial_params, method='BFGS')
        return result.x
    
    def inverse_mills_ratio(self, W, params):
        z = np.dot(W, params)
        return norm.pdf(z) / (norm.cdf(z) + self.epsilon)
    
    def estimate_did(self):
        # First stage: Estimate selection equation
        selection_params = self.estimate_selection()
        
        # Compute Inverse Mills Ratio
        imr = self.inverse_mills_ratio(self.W, selection_params)
        
        # Augment X with IMR
        X_augmented = np.column_stack((self.X, imr))
        
        # Second stage: Estimate DiD model
        DT = self.D * self.T
        X_full = np.column_stack((np.ones_like(self.D), self.D, self.T, DT, X_augmented))
        
        # OLS estimation
        beta = np.linalg.inv(X_full.T @ X_full) @ X_full.T @ self.Y
        
        # Compute standard errors (simplified, ignoring the first stage estimation)
        residuals = self.Y - X_full @ beta
        sigma2 = np.mean(residuals**2)
        var_beta = sigma2 * np.linalg.inv(X_full.T @ X_full)
        se_beta = np.sqrt(np.diag(var_beta))
        
        return beta, se_beta

In [9]:

# Function to simulate data
def simulate_data(n_samples, n_covariates, treatment_effect, selection_strength):
    np.random.seed(42)
    
    # Covariates
    X = np.random.randn(n_samples, n_covariates)
    W = np.column_stack((np.ones(n_samples), X))
    
    # Treatment assignment (selection equation)
    selection_params = np.random.randn(W.shape[1]) * selection_strength
    p_treatment = norm.cdf(np.dot(W, selection_params))
    D = (np.random.rand(n_samples) < p_treatment).astype(int)
    
    # Time periods
    T = np.random.binomial(1, 0.5, n_samples)
    
    # Outcome
    epsilon = np.random.randn(n_samples)
    Y = 1 + 0.5 * D + 0.5 * T + treatment_effect * D * T + np.dot(X, np.random.randn(n_covariates)) + epsilon
    
    return Y, D, T, X, W


In [10]:

# Simulate data
n_samples = 10000
n_covariates = 3
true_treatment_effect = 2
selection_strength = 0.5

Y, D, T, X, W = simulate_data(n_samples, n_covariates, true_treatment_effect, selection_strength)


In [11]:

# Estimate DiD
estimator = DIDEstimator(Y, D, T, X, W)
beta, se_beta = estimator.estimate_did()


In [12]:

# Print results
param_names = ['Intercept', 'D', 'T', 'D*T (Treatment Effect)'] + [f'X{i+1}' for i in range(n_covariates)] + ['IMR']
for name, b, se in zip(param_names, beta, se_beta):
    print(f"{name}: {b:.4f} (SE: {se:.4f})")

print(f"\nTrue Treatment Effect: {true_treatment_effect}")
print(f"Estimated Treatment Effect: {beta[3]:.4f} (SE: {se_beta[3]:.4f})")

Intercept: 1.3956 (SE: 0.4710)
D: 0.4683 (SE: 0.0367)
T: 0.4936 (SE: 0.0226)
D*T (Treatment Effect): 2.0384 (SE: 0.0502)
X1: 0.4097 (SE: 0.1280)
X2: -0.3348 (SE: 0.0731)
X3: -0.2260 (SE: 0.0207)
IMR: -0.2553 (SE: 0.3079)

True Treatment Effect: 2
Estimated Treatment Effect: 2.0384 (SE: 0.0502)
